# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
cities_csv = "../output_data/cityweather_data.csv"
city_df = pd.read_csv(cities_csv)
city_df.head()


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,orbetello,42.4408,11.2196,49.80,93.0,0.0,3.44,IT,1.643578e+09
1,1,san patricio,28.0170,-97.5169,72.32,50.0,28.0,11.65,US,1.643578e+09
2,2,cozumel,20.5083,-86.9458,74.95,57.0,40.0,3.00,MX,1.643578e+09
3,3,yibin,28.7667,104.6238,42.91,84.0,85.0,2.17,CN,1.643578e+09
4,5,ilulissat,69.2167,-51.1000,6.82,56.0,75.0,14.97,GL,1.643578e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [13]:
gmaps.configure(g_key)
locations = city_df[["Lat", "Lng"]]
humidity = city_df['Humidity'].astype(int)

fig = gmaps.figure(center=(0,0), zoom_level=1)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=25,
                                 point_radius=1)

In [14]:
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [15]:
vacation_city_df = city_df.copy()
vacation_city_df = vacation_city_df.loc[(vacation_city_df['Max Temp'] >= 70)&(vacation_city_df['Max Temp'] <= 80)&(vacation_city_df['Wind Speed'] <= 10)&(vacation_city_df['Cloudiness'] == 0)]
vacation_city_df = vacation_city_df.dropna()
vacation_city_df = vacation_city_df.reset_index()
vacation_city_df

,index,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,30,32,vicuna,-30.0319,-70.7081,79.77,43.0,0.0,9.04,CL,1.643578e+09
1,196,214,sur,22.5667,59.5289,71.47,65.0,0.0,2.84,OM,1.643578e+09
2,236,256,solana beach,32.9912,-117.2711,78.89,46.0,0.0,5.01,US,1.643578e+09
3,316,345,quelimane,-17.8786,36.8883,78.75,89.0,0.0,2.30,MZ,1.643578e+09
4,338,371,hambantota,6.1241,81.1185,75.54,100.0,0.0,2.30,LK,1.643578e+09
5,417,458,twentynine palms,34.1356,-116.0542,72.34,7.0,0.0,1.01,US,1.643578e+09
6,519,570,sun city west,33.6620,-112.3413,73.85,22.0,0.0,3.00,US,1.643579e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [16]:
hotel_df=vacation_city_df.copy()
hotel_df = hotel_df.drop(columns=['index','Unnamed: 0'])
hotel_df["Hotel Name"]=''
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,vicuna,-30.0319,-70.7081,79.77,43.0,0.0,9.04,CL,1.643578e+09,
1,sur,22.5667,59.5289,71.47,65.0,0.0,2.84,OM,1.643578e+09,
2,solana beach,32.9912,-117.2711,78.89,46.0,0.0,5.01,US,1.643578e+09,
3,quelimane,-17.8786,36.8883,78.75,89.0,0.0,2.30,MZ,1.643578e+09,
4,hambantota,6.1241,81.1185,75.54,100.0,0.0,2.30,LK,1.643578e+09,
5,twentynine palms,34.1356,-116.0542,72.34,7.0,0.0,1.01,US,1.643578e+09,
6,sun city west,33.6620,-112.3413,73.85,22.0,0.0,3.00,US,1.643579e+09,


In [31]:
hotels=[]
x=0
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
for cities in hotel_df["City"]:
    coordinates=str(hotel_df['Lat'][x]) + "," + str(hotel_df['Lng'][x])
    x+=1
    radius=5000
    target="lodging"
    params = {
    "location": coordinates,
    "radius": radius,
    "type": target,
    "key": g_key
    }
    
    try:
        response = requests.get(base_url, params=params)
        hotel_results = response.json()
        hotels.append(hotel_results['results'][0]['name'])

    except IndexError:
        hotels.append("None")
        
        
hotel_df['Hotel Name'] = hotels
hotel_df    

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,vicuna,-30.0319,-70.7081,79.77,43.0,0.0,9.04,CL,1.643578e+09,Los Mosaicos
1,sur,22.5667,59.5289,71.47,65.0,0.0,2.84,OM,1.643578e+09,Sur Plaza Hotel
2,solana beach,32.9912,-117.2711,78.89,46.0,0.0,5.01,US,1.643578e+09,Hilton San Diego/Del Mar
3,quelimane,-17.8786,36.8883,78.75,89.0,0.0,2.30,MZ,1.643578e+09,Hotel Flamingo
4,hambantota,6.1241,81.1185,75.54,100.0,0.0,2.30,LK,1.643578e+09,Bungalow 63
5,twentynine palms,34.1356,-116.0542,72.34,7.0,0.0,1.01,US,1.643578e+09,Holiday Inn Express & Suites Twentynine Palms-...
6,sun city west,33.6620,-112.3413,73.85,22.0,0.0,3.00,US,1.643579e+09,None


In [33]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [35]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig.add_layer(heat_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))